In [1]:
import time
import DoS_scraper
from DoS_scraper import get_allnews_urls
from DoS_scraper import parse_page
from DoS_scraper import get_soup

DoS_urls = get_allnews_urls(begin_year=2018, end_year=2019, verbose=True)
print('%d urls for all news' % len(DoS_urls))


get briefing statement urls 2018 / 2019
1790 urls for all news


In [9]:
from DoS_scraper import get_soup
from DoS_scraper import parse_page
soup = get_soup('https://www.state.gov/r/pa/prs/ps/2019/02/288828.htm')


In [15]:
parse_page('https://www.state.gov/r/pa/prs/ps/2019/02/288818.htm')

{'title': 'List of Participants: Meeting of the Global Coalition To Defeat ISIS',
 'time': 'February 6, 2019',
 'content': '\nAFGHANISTAN\n\n\n Her Excellency Roya Rahmani\n Ambassador of Afghanistan to the United States\n\n\n \n\n\nALBANIA\n\n\n His Excellency Gent Cakaj\n Acting Minister for Europe and Foreign Affairs\n\n\n \n\n\nAUSTRALIA\n\n\n Mr. Anthony Sheehan\nDeputy Secretary, Department of Foreign Affairs and Trade\n\n\n \n\n\nAUSTRIA\n\n\n Mr. Gerhard Alois Maynhardt\nMinister Plenipotentiary, Federal Ministry for Europe, Integration, and Foreign Affairs\n\n\n \n\n\nBAHRAIN\n\n\n His Excellency Khalid Ahmed Al Khalifa\n Minister of Foreign Affairs\n\n\n \n\n\nBELGIUM\n\n\n His Excellency Axel Kenes\n Vice Minister for Political Affairs\n\n\n \n\n\nBOSNIA AND HERZEGOVINA\n\n\n His Excellency Igor Crnadak\n Minister of Foreign Affairs\n\n\n \n\n\nBULGARIA\n\n\n His Excellency Tihomir Stoytchev\nAmbassador of Bulgaria to the United States\n\n\n \n\n\nCAMEROON\n\n\n His Excellen

In [14]:
parse_page('https://www.state.gov/r/pa/prs/ps/2019/02/288777.htm')

{'title': 'U.S. Department of State Launches 22.33: A Podcast of Life-Changing Stories',
 'time': 'February 5, 2019',
 'content': '\nThe U.S. Department of State has launched a new podcast, 22.33, that discusses the impact of State Department-sponsored international exchange programs on individuals and communities around the world. Each episode will highlight personal stories from American and foreign exchange program alumni who left home to tackle shared global challenges with their counterparts overseas. These experiences have created ripple effects that continue to influence participants and entire communities.\n\n\nNew episodes of the podcast will be released each Friday throughout 2019. The Collaboratory, an innovation space within the State Department’s Bureau of Educational and Cultural Affairs, has created 22.33 to illustrate the power of exchange programs to meet U.S. foreign policy objectives, told through the stories of individuals. The podcast’s namesake comes from Title 22

In [2]:
def pprint(json_object):
    for k, v in json_object.items():
        print('{} : {} ..'.format(k, str(v)[:100]))
    print('\n')

SLEEP = 0.5

for url in DoS_urls[:5]:
    json_object = parse_page(url)
    pprint(json_object)

title : Attack in Giza ..
time : December 28, 2018 ..
content : 
The United States strongly condemns the attack carried out today on a tourist bus in Giza. Our deep ..
url : https://www.state.gov/r/pa/prs/ps/2018/12/288285.htm ..
scrap_time : 2019-02-07-05 08:09:40 ..


title : Israel's Right to Self-Defense ..
time : December 28, 2018 ..
content : 
The United States fully supports Israel’s right to defend itself against Iranian regional actions t ..
url : https://www.state.gov/r/pa/prs/ps/2018/12/288282.htm ..
scrap_time : 2019-02-07-05 08:09:41 ..


title : Secretary Pompeo Travels to Brazil and Colombia To Strengthen Prosperity, Security, and Democracy ..
time : December 28, 2018 ..
content : 
Secretary Pompeo is leading a Presidential Delegation to Brazil for the inauguration of President-e ..
url : https://www.state.gov/r/pa/prs/ps/2018/12/288281.htm ..
scrap_time : 2019-02-07-05 08:09:41 ..


title : Senior State Department Official Previewing Secretary Pompeo's Upcoming Travel t

In [3]:
from DoS_scraper import yield_latest_allnews

begin_date = '2018-12-01'
max_num = 50
sleep = 1.0

for i, json_obj in enumerate(yield_latest_allnews(begin_date, max_num, sleep)):
    title = json_obj['title']
    time = json_obj['time']
    print('[{} / {}] ({}) {}'.format(i+1, max_num, time, title))

[1 / 50] (February 6, 2019) Additional Payments Under Holocaust Deportation Claims Program
[2 / 50] (February 6, 2019) Africa Working Group of the U.S.-Morocco Strategic Dialogue
[3 / 50] (February 6, 2019) Delivery of Humanitarian Assistance to Rukban
[4 / 50] (February 6, 2019) Grenada Independence Day
[5 / 50] (February 6, 2019) Interview With Maria Bartiromo of Mornings With Maria on Fox Business Network
[6 / 50] (February 6, 2019) Interview With Trish Regan of Trish Regan Primetime on Fox Business Network


KeyboardInterrupt: 